In [ ]:
import pandas as pd
from duke_dbt_data import dcmread_image
import pydicom as dicom
from typing import AnyStr, BinaryIO, Dict, List, NamedTuple, Optional, Union
import matplotlib.pyplot as plt
import numpy as np
import imageio
from PIL import Image, ImageOps
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)
from mmcv import Config
import mmcv
from mmdet.apis import set_random_seed
# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv.runner import load_checkpoint
from mmdet.apis import multi_gpu_test, single_gpu_test
print(get_compiling_cuda_version())
print(get_compiler_version())
from mmdet.datasets import build_dataset, build_dataloader
from mmdet.models import build_detector
from mmdet.apis import train_detector
import copy
import os.path as osp
from mmcv.parallel import MMDataParallel
import mmcv
import numpy as np
import json
import os
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from PIL import Image

In [ ]:
# read data path
basic_path =  'the path to save dicom images' # the path to save the images
csv_path ='./data_csv/'# the path to svae the labels and tables
data_type ="test" # or val or test
path_table = pd.DataFrame(pd.read_csv(csv_path +'BCS-DBT file-paths-'+ data_type + '.csv'))
path_list = path_table['descriptive_path']
view_list = path_table['View']
target_list = basic_path + path_list

In [ ]:
# load the model
cfg = Config.fromfile('../../configs/breast/faster_rcnn_r50_fpn_2x_breast.py')
# Set up working dir to save files and logs.
cfg.work_dir = '../../data/work_dirs/faster_rcnn_r50_fpn_2x_breast'
checkpoint = cfg.work_dir +'/latest.pth'
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
model.CLASSES = checkpoint['meta']['CLASSES']
model = MMDataParallel(model, device_ids=[2])
model.cfg = cfg
#result = inference_detector(model, img_slice_rgb)
#show_result_pyplot(model, img_slice_rgb, result,score_thr=0.5)

# reproduce the final results table

In [ ]:
df = pd.DataFrame(columns = ['PatientID','StudyUID','View','X','Width','Y','Height','Z','Depth','Score'])
conf_thre = 0.1
line_cnt = 0
for i in range(len(target_list)):
    # get the side of this image, the first letter
    side = view_list[i][0]
    # read dicom image
    img = dcmread_image(fp = target_list[i], view = view_list[i])
    patient_id = path_table['PatientID'][i]
    study_id = path_table['StudyUID'][i]
    view = path_table['View'][i]
    for slice_now in range(img.shape[0]):
        img_slice = img[slice_now]
        if side == 'r':
            img_slice = img_slice[:,::-1]
        img_slice = np.array(img_slice*255.0/65535.0, dtype=np.uint8)
        img_slice_rgb = np.zeros((img_slice.shape[0],img_slice.shape[1],3),dtype=np.uint8)
        img_slice_rgb[:,:,0] = img_slice_rgb[:,:,1] = img_slice_rgb[:,:,2] = img_slice
        result = inference_detector(model, img_slice_rgb)[0]
        for det_box in result:
            conf_score = det_box[-1]
            if conf_score>conf_thre:
                line_cnt +=1
                X = int(det_box[0])
                Y = int(det_box[1])
                Width = int(det_box[2] - det_box[0])
                Height = int(det_box[3] - det_box[1])
                if side == 'r':
                    X = img_slice.shape[1]-1-X-Width
                df.loc[line_cnt] = [patient_id, study_id, view, X,Width, Y,Height,slice_now,-1,conf_score]
    df.to_csv(r'faster_rcnn_test_table_new.csv',index = False, header=True)